In [2]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import text
from sqlalchemy import create_engine, func
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import hvplot.pandas
# Create an engine
engine = create_engine('sqlite:///mydatabase.db')

# Read the CSV file into a DataFrame
df = pd.read_csv('Resources/cleaned_housing_market_data.csv')

# Load the DataFrame into the database
df.to_sql('Housing_Data', con=engine, if_exists='replace', index=False)
Session = sessionmaker(bind=engine)
session = Session()

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [3]:
query_1 = 'SELECT * FROM Housing_Data'

In [4]:
result = session.execute(text(query_1))

df_result= pd.DataFrame(result)
df_result.head()
#for row in result:
 #   print(row)


,Index,Year,Month,County Name,Active Listing Count,Median Days on Market,New Listing Count,Median List Price Per Sqft,Median Sqft,Avg Listing Price,Total Listing Count
0,0,2024,10,"fulton, in",45.0,54.0,14.0,146.0,2102.0,363246.0,61.0
1,1,2024,10,"brooks, ga",25.0,60.0,4.0,139.0,1729.0,295232.0,26.0
2,2,2024,10,"worcester, ma",1104.0,31.0,874.0,286.0,1985.0,635754.0,1361.0
3,3,2024,10,"sussex, va",18.0,28.0,8.0,169.0,1764.0,379463.0,25.0
4,4,2024,10,"clark, wa",1474.0,58.0,604.0,309.0,2225.0,847493.0,2292.0


In [5]:
#Scaling Data
# Scaling the numeric columns
housing_data_scaled = StandardScaler().fit_transform(df_result[["Active Listing Count","Median Days on Market","New Listing Count","Median List Price Per Sqft","Median Sqft","Avg Listing Price",
                                                                  "Total Listing Count"]])

# Creating a DataFrame with with the scaled data
df_housing_transformed = pd.DataFrame(housing_data_scaled, columns=["Active Listing Count","Median Days on Market","New Listing Count","Median List Price Per Sqft","Median Sqft","Avg Listing Price",
                                                                  "Total Listing Count"])

# Display sample data
df_housing_transformed.head()

,Active Listing Count,Median Days on Market,New Listing Count,Median List Price Per Sqft,Median Sqft,Avg Listing Price,Total Listing Count
0,-0.339173,-0.566026,-0.344828,0.040579,0.550826,-0.020479,-0.340977
1,-0.359953,-0.378974,-0.366936,-0.026694,-0.423458,-0.214969,-0.365619
2,0.761115,-1.283059,1.556485,1.386050,0.245219,0.758776,0.574306
3,-0.367226,-1.376586,-0.358093,0.261621,-0.332037,0.025895,-0.366324
4,1.145540,-0.441324,0.959561,1.607092,0.872104,1.364258,1.229789


In [6]:
#encoding with get dummies
county_encoded = pd.get_dummies(df_result["County Name"], columns=['County Name'])
county_encoded

abbeville, sc  acadia, la  accomack, va  ada, id  adair, ia  \
0               False       False         False    False      False   
1               False       False         False    False      False   
2               False       False         False    False      False   
3               False       False         False    False      False   
4               False       False         False    False      False   
...               ...         ...           ...      ...        ...   
220761          False       False         False    False      False   
220762          False       False         False    False      False   
220763          False       False         False    False      False   
220764          False       False         False    False      False   
220765          False       False         False    False      False   

        adair, ky  adair, mo  adair, ok  adams, co  adams, ia  adams, id  \
0           False      False      False      False      False      False   
1           False      False      False      False      False      False   
2           False      False      False      False      False      False   
3           False      False      False      False      False      False   
4           False      False      False      False      False      False   
...           ...        ...        ...        ...        ...        ...   
220761      False      False      False      False      False      False   
220762      False      False      False      False      False      False   
220763      False      False      False      False      False      False   
220764      False      False      False      False      False      False   
220765      False      False      False      False      False      False   

        adams, il  adams, in  adams, ms  adams, nd  adams, ne  adams, oh  \
0           False      False      False      False      False      False   
1           False      False      False      False      False      False   
2           False      False      False      False      False      False   
3           False      False      False      False      False      False   
4           False      False      False      False      False      False   
...           ...        ...        ...        ...        ...        ...   
220761      False      False      False      False      False      False   
220762      False      False      False      False      False      False   
220763      False      False      False      False      False      False   
220764      False      False      False      False      False      False   
220765      False      False      False      False      False      False   

        adams, pa  adams, wa  adams, wi  addison, vt  aiken, sc  aitkin, mn  \
0           False      False      False        False      False       False   
1           False      False      False        False      False       False   
2           False      False      False        False      False       False   
3           False      False      False        False      False       False   
4           False      False      False        False      False       False   
...           ...        ...        ...          ...        ...         ...   
220761      False      False      False        False      False       False   
220762      False      False      False        False      False       False   
220763      False      False      False        False      False       False   
220764      False      False      False        False      False       False   
220765      False      False      False        False      False       False   

        alachua, fl  alamance, nc  alameda, ca  alamosa, co  albany, ny  \
0             False         False        False        False       False   
1             False         False        False        False       False   
2             False         False        False        False       False   
3             False         False        False        False       False   
4    

In [7]:
date_df = pd.DataFrame({
    "Year": df_result["Year"],
    "Month": df_result["Month"]
})
date_df

,Year,Month
0,2024,10
1,2024,10
2,2024,10
3,2024,10
4,2024,10
...,...,...
220761,2016,7
220762,2016,7
220763,2016,7
220764,2016,7


In [8]:
#concatinating 
combined_df = pd.concat([date_df, df_housing_transformed], axis=1)
combined_df.head()

,Year,Month,Active Listing Count,Median Days on Market,New Listing Count,Median List Price Per Sqft,Median Sqft,Avg Listing Price,Total Listing Count
0,2024,10,-0.339173,-0.566026,-0.344828,0.040579,0.550826,-0.020479,-0.340977
1,2024,10,-0.359953,-0.378974,-0.366936,-0.026694,-0.423458,-0.214969,-0.365619
2,2024,10,0.761115,-1.283059,1.556485,1.386050,0.245219,0.758776,0.574306
3,2024,10,-0.367226,-1.376586,-0.358093,0.261621,-0.332037,0.025895,-0.366324
4,2024,10,1.145540,-0.441324,0.959561,1.607092,0.872104,1.364258,1.229789


In [9]:
#reshape
#X = combined_df[['Median Sqft', 'Median List Price Per Sqft', 'Year']]  
#y = combined_df['Avg Listing Price'] 
#X = X.values.reshape(-1, 1)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

X = combined_df[['Median Sqft', 'Median List Price Per Sqft', 'Year']]
y = combined_df['Avg Listing Price'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(176612, 3) (44154, 3) (176612,) (44154,)


In [10]:
# Create a model with scikit-learn
model = LinearRegression()

# Fit the data into the model
model.fit(X, y)


LinearRegression()

In [11]:
# Make predictions using the X set
predicted_y_values = model.predict(X)

In [12]:
# Create a copy of the original data
df_predicted = combined_df.copy()

# Add a column with the predicted salary values
df_predicted["Avg_median_predicted"] = predicted_y_values

# Display sample data
df_predicted.head()

,Year,Month,Active Listing Count,Median Days on Market,New Listing Count,Median List Price Per Sqft,Median Sqft,Avg Listing Price,Total Listing Count,Avg_median_predicted
0,2024,10,-0.339173,-0.566026,-0.344828,0.040579,0.550826,-0.020479,-0.340977,0.102844
1,2024,10,-0.359953,-0.378974,-0.366936,-0.026694,-0.423458,-0.214969,-0.365619,-0.127691
2,2024,10,0.761115,-1.283059,1.556485,1.386050,0.245219,0.758776,0.574306,1.168288
3,2024,10,-0.367226,-1.376586,-0.358093,0.261621,-0.332037,0.025895,-0.366324,0.128726
4,2024,10,1.145540,-0.441324,0.959561,1.607092,0.872104,1.364258,1.229789,1.464613


In [13]:
#Formula for the case of three variables.
#print(f"Model's formula: y = {model.intercept_} + {model.coef_[0]x_1 + {model.coef_[1]}x_2 + {model.coef_[2]}x_3")

In [14]:
from sklearn.metrics import r2_score

In [15]:
r2_score( combined_df['Avg Listing Price'] ,predicted_y_values)

0.7445084964984598